In [71]:
from typing import List, Union
import mido

TICKS_PER_BEAT = 480
TIME_SIG = (3, 4)
INSERT_ZERO = 480

NOTE_NAME = ['1', '#1', '2', '#2', '3', '4', '#4', '5', '#5', '6', '#6', '7']


BEAT4 = TICKS_PER_BEAT
BEAT8 = TICKS_PER_BEAT // 2
BEAT16 = TICKS_PER_BEAT // 4


match TIME_SIG[1]:
    case 4:
        SEP = BEAT4 * TIME_SIG[0]
    case 8:
        SEP = BEAT8 * TIME_SIG[0]
    case 16:
        SEP = BEAT16 * TIME_SIG[0]
    case _:
        raise

from enum import Enum
class NoteTimeValue(Enum):
    CROTCHET = ''
    QUAVER = 'q'
    SEMIQUAVER = 's'


from dataclasses import dataclass

@dataclass
class Note:
    tone: int
    length: int
    time: NoteTimeValue
    dotted: int = 0
    dash: int = 0
    slur: str = ''

    def __repr__(self) -> str:
        offset, index = ((self.tone-60) // 12, (self.tone-60) % 12) if self.tone != 0 else (0, None)
        oct = abs(offset) * ("'" if offset > 0 else ',')
        note_name = NOTE_NAME[index] if index is not None else '0'
        return self.time.value + note_name + oct + ('.' * self.dotted) + (f" {self.slur}" if self.tone != 0 and self.slur else '') + (' -' * self.dash)


class NoteGroup:
    def __init__(self, notes: List[Union[Note, 'NoteGroup']]) -> None:
        self.notes = []
        for note in notes:
            if isinstance(note, NoteGroup):
                self.notes.extend(note.notes)
            else:
                self.notes.append(note)
        self.set_slurs()
    def set_slurs(self):
        if len(self.notes) < 2:
            return
        self.notes[0].slur = '('
        self.notes[-1].slur = ')'
        for note in self.notes[1:-1]:
            note.slur = ''
    
    def __repr__(self) -> str:
        return ' '.join([str(i) for i in self.notes])

def process_message(tone: int, length: int, limit: int = None):
    if limit is not None and length > limit:
        return NoteGroup([process_message(tone, limit), process_message(tone, length - limit, SEP)])


    if length == 0:
        return NoteGroup([])
    assert length % BEAT16 == 0
    dotted = 0
    match length // BEAT16:
        case 1:
            time = NoteTimeValue.SEMIQUAVER
        case 2:
            time = NoteTimeValue.QUAVER
        case 3:
            time = NoteTimeValue.QUAVER
            dotted = 1
        case 4:
            time = NoteTimeValue.CROTCHET
        case 5:
            return NoteGroup([process_message(tone, BEAT4), process_message(tone, BEAT16)])
        case 6:
            time = NoteTimeValue.CROTCHET
            dotted = 1
        case 7:
            time = NoteTimeValue.CROTCHET
            dotted = 2
        case _:  # val >= 8
            extra = length % BEAT4
            dash = (length - extra) // BEAT4 - 1
            n = process_message(tone, BEAT4 + extra)
            if isinstance(n, NoteGroup):
                n.notes[0].dash = dash
                n.notes[0].length += dash * BEAT4
            else:
                n.dash = dash
                n.length += dash * BEAT4
            return n

    return Note(tone, length, time, dotted)

                

midi = mido.MidiFile('res.mid')

track = []

assert midi.ticks_per_beat == TICKS_PER_BEAT
sep = -INSERT_ZERO
for i in range(len(midi.tracks[0])-2):
    msg1: mido.messages.messages.Message = midi.tracks[0][i]
    msg2: mido.messages.messages.Message = midi.tracks[0][i + 1]
    note_time: int = msg2.time
    note_type: str = msg1.type
    if note_type == 'note_off':
        note_type = 'note_on'
        msg1.velocity = 0
    if note_type != 'note_on':
        continue
    if note_time < BEAT16:
        continue
    if msg1.velocity == 0:
        msg1.note = 0

    left_time = (sep - BEAT16) % SEP + BEAT16
    track.append(process_message(msg1.note, note_time, left_time))
    sep -= note_time
track.insert(0, process_message(0, INSERT_ZERO, SEP))
track.append(process_message(0, (sep - BEAT16) % SEP + BEAT16, SEP))
        
# track
# t = 0
# for note in NoteGroup(track).notes:
#     print(note)
#     t += note.length
#     if t == SEP:
#         print('------')
#         t = 0
#         continue
#     if t > SEP:
#         raise

# from pyperclip import copy
# copy(' '.join(str(i) for i in NoteGroup(track).notes))

res = []
for note in track:
    if isinstance(note, NoteGroup):
        for i in note.notes:
            res.append(str(i))
    else:
        res.append(note)

copy(' '.join(str(i) for i in res))


In [67]:
midi = mido.MidiFile('res.mid')
list([getattr(a, 'time', None) for a in midi.tracks[0]])
midi.tracks[0][160:]
midi

MidiFile(type=1, ticks_per_beat=480, tracks=[
  MidiTrack([
    Message('program_change', channel=0, program=41, time=0),
    MetaMessage('set_tempo', tempo=500000, time=0),
    Message('note_on', channel=0, note=66, velocity=64, time=0),
    Message('note_off', channel=0, note=66, velocity=64, time=480),
    Message('note_on', channel=0, note=67, velocity=64, time=0),
    Message('note_off', channel=0, note=67, velocity=64, time=480),
    Message('note_on', channel=0, note=59, velocity=64, time=0),
    Message('note_off', channel=0, note=59, velocity=64, time=480),
    Message('note_on', channel=0, note=67, velocity=64, time=0),
    Message('note_off', channel=0, note=67, velocity=64, time=480),
    Message('note_on', channel=0, note=66, velocity=64, time=0),
    Message('note_off', channel=0, note=66, velocity=64, time=720),
    Message('note_on', channel=0, note=64, velocity=64, time=240),
    Message('note_off', channel=0, note=64, velocity=64, time=240),
    Message('note_on', cha